# Finetuning on restaurant reviews
This notebook finetunes a custom model on restaurant reviews. The goal is to build an agentic RAG application which can better understand and retrieve queries about food. 


### Dependencies and Boilerplate

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [21]:
import random
import os
import numpy as np
import torch
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [22]:
seed_everything(42)

In [23]:
from dotenv import load_dotenv
_ = load_dotenv()


In [ ]:
# Not needed unless on colab
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters ragas==0.2.10 rapidfuzz jq


In [ ]:
!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml


In [16]:
!pip install jq

In [27]:

import os
from langchain_community.document_loaders import HuggingFaceDatasetLoader, JSONLoader, CSVLoader
DATA_ROOT = "../../data/10000_restaurant_reviews/restaurant_reviews.csv"
dataloader = CSVLoader(DATA_ROOT)

In [28]:
documents = dataloader.load()

In [29]:
#shuffle the documents
import numpy as np
documents = np.random.permutation(documents)

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len
)

In [31]:
documents = text_splitter.split_documents(documents)


In [32]:
len(documents)

10882

In [33]:
# Take a smaller subset of documents for training
training_documents = documents[:1000]

In [13]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [14]:
# Create documentation splits
training_split_documents = training_documents[:len(training_documents) - 200]
val_split_documents = training_documents[len(training_documents) - 200:len(training_documents)-100]
test_split_documents = training_documents[len(training_documents)-100:]

## Construct Fine-tuning Dataset

In [19]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

In [20]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [21]:
question_generation_chain = qa_prompt_template | qa_chat_model


In [86]:
import tqdm
import asyncio

"""
Sample Usage of TQDM:

for i in tqdm.tqdm(range(10)):
  time.sleep(1)
"""

async def create_questions(documents, n_questions):

    questions = {}
    relevant_docs = {}
    question_pattern = r"^\d+\.\s(.*)$"
    id_set = set()
    ### YOUR CODE HERE
    for i, doc in tqdm.tqdm(enumerate(documents)):
        r = question_generation_chain.invoke({'n_questions': n_questions, "context": doc.page_content})
        import re
        import uuid
        question_list = re.findall(question_pattern, r.content, re.MULTILINE)
        for question in question_list:
            id = uuid.uuid4()
            while id in id_set:
                id = uuid()
            id_set.add(id)
            questions[str(id)] = question
            if id not in relevant_docs:
                relevant_docs[str(id)] = [str(doc.metadata["id"])]
            else:
                relevant_docs[id].append(str(doc.metadata["id"]))

    return questions, relevant_docs

In [87]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)


800it [15:46,  1.18s/it]


In [88]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)


100it [01:41,  1.02s/it]


In [89]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)


100it [01:39,  1.01it/s]


In [94]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [95]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [96]:
test_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : test_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [15]:
import json
with open("training_dataset.jsonl", "r") as f:
  train_dataset = json.load(f)

with open("val_dataset.jsonl", "r") as f:
  val_dataset = json.load(f)

with open("test_dataset.jsonl", "r") as f:
  test_dataset = json.load(f)

### Fine-tuning `snowflake-arctic-embed-l`

In [ ]:
!pip install -qU sentence_transformers datasets pyarrow


In [99]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

In [100]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [101]:
BATCH_SIZE = 10


In [102]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [103]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [104]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [105]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [106]:
EPOCHS = 10


In [107]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [108]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/Users/deman/Dev/Maven/MavenAIBootcamp6/AIE6-Certification-Challenge/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,No log,0.865000,0.935000,0.950000,0.980000,0.865000,0.311667,0.190000,0.098000,0.865000,0.935000,0.950000,0.980000,0.923887,0.905857,0.907032
100,No log,No log,0.885000,0.935000,0.955000,0.985000,0.885000,0.311667,0.191000,0.098500,0.885000,0.935000,0.955000,0.985000,0.931342,0.914635,0.915857
150,No log,No log,0.880000,0.950000,0.960000,0.975000,0.880000,0.316667,0.192000,0.097500,0.880000,0.950000,0.960000,0.975000,0.930701,0.916139,0.918109
160,No log,No log,0.885000,0.945000,0.955000,0.970000,0.885000,0.315000,0.191000,0.097000,0.885000,0.945000,0.955000,0.970000,0.930119,0.917042,0.919404
200,No log,No log,0.880000,0.955000,0.975000,0.990000,0.880000,0.318333,0.195000,0.099000,0.880000,0.955000,0.975000,0.990000,0.938205,0.921298,0.922109
250,No log,No log,0.900000,0.955000,0.980000,0.990000,0.900000,0.318333,0.196000,0.099000,0.900000,0.955000,0.980000,0.990000,0.945418,0.931000,0.931871
300,No log,No log,0.900000,0.975000,0.985000,0.990000,0.900000,0.325000,0.197000,0.099000,0.900000,0.975000,0.985000,0.990000,0.950134,0.936667,0.937415
320,No log,No log,0.910000,0.975000,0.980000,0.990000,0.910000,0.325000,0.196000,0.099000,0.910000,0.975000,0.980000,0.990000,0.953234,0.941000,0.941871
350,No log,No log,0.895000,0.980000,0.985000,0.995000,0.895000,0.326667,0.197000,0.099500,0.895000,0.980000,0.985000,0.995000,0.950081,0.935083,0.935346
400,No log,No log,0.900000,0.985000,0.985000,0.995000,0.900000,0.328333,0.197000,0.099500,0.900000,0.985000,0.985000,0.995000,0.955882,0.942500,0.942750


In [109]:
hf_username = "deman539"
import uuid

model.push_to_hub(f"{hf_username}/food-review-ft-snowflake-l-{uuid.uuid4()}")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/deman539/food-review-ft-snowflake-l-f18eeff6-7504-48c7-af10-1d2d85ca8caa/commit/b8baf52b6a8fb5105ec17c9384eb54be2efea980'

### Evaluation

In [110]:
# Creating a new set of documents to do a quick evaluation on
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 100,
    length_function = len
)

training_documents = text_splitter.split_documents(documents)

In [111]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

In [112]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

#### OpenAI Evaluation

In [113]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 200/200 [01:27<00:00,  2.29it/s]


In [114]:
te3_results_df = pd.DataFrame(te3_results)


In [117]:
te3_hit_rate = te3_results_df["is_hit"].mean()
print(f"Hit rate for openai embeddings: {te3_hit_rate*100:.2f}%")

Hit rate for openai embeddings: 96.50%


#### Base Snowflake-Arctic-l Evaluation

In [118]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 200/200 [00:20<00:00,  9.54it/s]


In [119]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)


In [120]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
print(f"Hit rate for arctic-embed-m embeddings: {arctic_embed_m_hit_rate*100:.2f}%")

Hit rate for arctic-embed-m embeddings: 78.00%


#### Finetuned Snowflake-Arctic-L Evaluation

In [121]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 200/200 [00:11<00:00, 18.14it/s]


In [122]:
finetune_results_df = pd.DataFrame(finetune_results)


In [123]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
print(f"Hit rate for finetuned embeddings: {finetune_hit_rate*100:.2f}%")

Hit rate for finetuned embeddings: 97.50%


Overall, the finetuned embeddings (on this very basic test) perform better than OpenAI Embeddings

### RAGAS Evaluation

In [126]:
!pip install rapidfuzz==3.13.0 ragas==0.2.10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached pysbd-0.3.4-py3-none-any.whl (71 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)


In [38]:
docs = documents[1000:2000]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(docs)

In [39]:
from ragas.testset import TestsetGenerator

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(training_documents, testset_size=100)

Applying SummaryExtractor:   0%|          | 0/465 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/1404 [00:00<?, ?it/s]

Node fc930fdb-50a0-4a08-902a-11e975f9d780 does not have a summary. Skipping filtering.
Node f4320702-b58b-43da-a254-71b959274ff2 does not have a summary. Skipping filtering.
Node aca4c4e9-2d7f-4ed6-b43a-c20245cfe0a8 does not have a summary. Skipping filtering.
Node 6dadac3f-542c-4377-8cd6-f1c7e1a1113c does not have a summary. Skipping filtering.
Node 50757393-1f8c-4b57-b4a1-7c49a11e1fca does not have a summary. Skipping filtering.
Node fc524aec-f2c3-4412-9688-6775ca35bf1d does not have a summary. Skipping filtering.
Node 6af99030-d243-4b7a-b0b6-f59c52943ff8 does not have a summary. Skipping filtering.
Node f549a377-3d42-4089-860b-7244c07ec208 does not have a summary. Skipping filtering.
Node 71f5e218-9a86-4109-819c-56dab9925019 does not have a summary. Skipping filtering.
Node 8c016ada-1837-4e59-beed-2305d84222cb does not have a summary. Skipping filtering.
Node b2d51896-d9ce-47bf-b723-d2909dec482e does not have a summary. Skipping filtering.
Node 44a702dc-597d-4327-8497-58447fe6ffe2 d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/3189 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/100 [00:00<?, ?it/s]

In [40]:
dataset.to_pandas()


,user_input,reference_contexts,reference,synthesizer_name
0,Who is Appalanaidu Allu in the context of the ...,[Restaurant: Mohammedia Shawarma\nReviewer: Ap...,Appalanaidu Allu is the reviewer for the resta...,single_hop_specifc_query_synthesizer
1,What you think about T Grill food hygiene and ...,[Restaurant: T Grill\nReviewer: Akshara\nRevie...,The review states that the hygiene level at T ...,single_hop_specifc_query_synthesizer
2,Is Mathura Vilas a good place for vegetarian f...,[Restaurant: Mathura Vilas\nReviewer: Hari\nRe...,"Yes, Mathura Vilas is described as a good plac...",single_hop_specifc_query_synthesizer
3,Can you tell me about the food hygiene standar...,[Restaurant: Delhi-39\nReviewer: Akshar Katari...,"According to a review by Akshar Kataria, the r...",single_hop_specifc_query_synthesizer
4,Why is American Wild Wings bad?,[Restaurant: American Wild Wings\nReviewer: Sw...,The review states that the reviewer ordered be...,single_hop_specifc_query_synthesizer
...,...,...,...,...
95,Indian desserts in indiranagar place good for ...,[<1-hop>\n\nRestaurant: Feast - Sheraton Hyder...,"In the indiranagar area, Feast at Sheraton Hyd...",multi_hop_specific_query_synthesizer
96,Wha is the role of Shubro in the Beyond Flavou...,[<1-hop>\n\nRestaurant: Beyond Flavours\nRevie...,Shubro is mentioned in both reviews of Beyond ...,multi_hop_specific_query_synthesizer
97,Based on the FOODGEEK review of Jonathan's Kit...,[<1-hop>\n\nRestaurant: Jonathan's Kitchen - H...,"A food enthusiast should be cautious, as the F...",multi_hop_specific_query_synthesizer
98,How do the reviews by Srinivas K and Srinivas ...,[<1-hop>\n\nRestaurant: Hyderabadi Daawat\nRev...,Srinivas Reddy's review of Hyderabadi Daawat w...,multi_hop_specific_query_synthesizer


In [41]:
# Base embedding model
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

In [42]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

In [44]:
rag_llm =  ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0
)

In [45]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [46]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="deman539/food-review-ft-snowflake-l-f18eeff6-7504-48c7-af10-1d2d85ca8caa")
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [51]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [48]:
for test_row in dataset:
  response = base_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [49]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [50]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))

In [52]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

base_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
base_result

Evaluating:   0%|          | 0/600 [00:00<?, ?it/s]

Exception raised in Job[308]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
Exception raised in Job[368]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
Exception raised in Job[392]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')


{'context_recall': 0.2998, 'faithfulness': 0.7608, 'factual_correctness': 0.3091, 'answer_relevancy': 0.3666, 'context_entity_recall': 0.2814, 'noise_sensitivity_relevant': 0.1437}

In [53]:
for test_row in dataset:
  response = finetune_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [54]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [55]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

finetune_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
finetune_result

Evaluating:   0%|          | 0/600 [00:00<?, ?it/s]

Exception raised in Job[152]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
Exception raised in Job[356]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
Exception raised in Job[392]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
Exception raised in Job[398]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
Exception raised in Job[494]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')


{'context_recall': 0.6383, 'faithfulness': 0.7685, 'factual_correctness': 0.4484, 'answer_relevancy': 0.6256, 'context_entity_recall': 0.5006, 'noise_sensitivity_relevant': 0.2411}

In [61]:
comparison_df = pd.DataFrame(
    [
     {'model': 'base','context_recall': 0.2998, 'faithfulness': 0.7608, 'factual_correctness': 0.3091, 'answer_relevancy': 0.3666, 'context_entity_recall': 0.2814, 'noise_sensitivity_relevant': 0.1437},
     {'model': 'finetuned', 'context_recall': 0.6383, 'faithfulness': 0.7685, 'factual_correctness': 0.4484, 'answer_relevancy': 0.6256, 'context_entity_recall': 0.5006, 'noise_sensitivity_relevant': 0.2411},

    ]
)

In [62]:
comparison_df

,model,context_recall,faithfulness,factual_correctness,answer_relevancy,context_entity_recall,noise_sensitivity_relevant
0,base,0.2998,0.7608,0.3091,0.3666,0.2814,0.1437
1,finetuned,0.6383,0.7685,0.4484,0.6256,0.5006,0.2411


It is very clear that the finetuned model performs much better on food related query retrieval.